# TP noté 2 L3 Algorithmique des graphes 2023-2024
# Durée: 2 heures

## Vous disposez uniquement des documents suivants :
- les transparents de cours en format pdf
- les fichiers `dictionnaireadjacenceoriente.py` et `dictionnaireadjacenceorientepondere.py`
- un accès à la documentation Python
- ce sujet du TP noté sous forme de Notebook Python (extension `.ipynb`)

Aucun autre document ou support n'est autorisé.

## Mettez tous vos fichiers et ce Notebook Python dans le répertoire EXAM
## Remplissez la première cellule avec vos noms et prénoms
## Indiquez si vous avez un tiers temps. 
## Répondez aux questions dans les cellules indiquées. 

Le code Python devra utiliser les classes `DictionnaireAdjacenceOrientePondere` et `DictionnaireAdjacencePondere`. Vos réponses devront fonctionner quel que soit le type des identifiants utilisés pour les sommets.

<font color=red>**Attention :**</font> ne modifiez pas les fichiers fournis. Ils seront réinitialisés lors de la correction. 


## Exercice 1: Flots

1. Écrivez une fonction prenant en paramètres un graphe orienté pondéré et un flot, et renvoyant le réseau résiduel associé. Le flot est donné sous la forme d'un dictionnaire associant à chaque arc une valeur, et satisfait les propriétés d'un flot que vous ne devez donc pas vérifier.

In [4]:
# corrigé exercice
def residuel(graphe, flot):
    resultat = type(graphe)()
    
    for u, v, capacite in graphe.arcs():
        if flot[(u, v)] < capacite:
            resultat.ajouter_arc(u, v, capacite - flot[(u, v)])
    
        if flot[(u, v)]:
            resultat.ajouter_arc(v, u, flot[(u, v)])
    
    return resultat
    
# tests -----------------------------------------------------------------------
from dictionnaireadjacenceorientepondere import DictionnaireAdjacenceOrientePondere as Graphe

# 1er exemple du cours, où 's' est la source et 't' le puits
G = Graphe()
G.ajouter_arcs([
    ('s', 'a', 16), ('s', 'b', 13), ('a', 'c', 12), ('b', 'a', 4 ), 
    ('b', 'd', 14), ('c', 'b', 9 ), ('c', 't', 20), ('d', 'c', 7 ), 
    ('d', 't', 4 )
])
flot = {
    ('s', 'a'): 11, ('s', 'b'): 8, ('a', 'c'): 12, ('b', 'a'): 1, 
    ('b', 'd'): 11, ('c', 'b'): 4, ('c', 't'): 15, ('d', 'c'): 7, 
    ('d', 't'): 4 
}
resultat = residuel(G, flot)
print(sorted(resultat.arcs()))

[('a', 'b', 1), ('a', 's', 11), ('b', 'a', 3), ('b', 'c', 4), ('b', 'd', 3), ('b', 's', 8), ('c', 'a', 12), ('c', 'b', 5), ('c', 'd', 7), ('c', 't', 5), ('d', 'b', 11), ('s', 'a', 5), ('s', 'b', 5), ('t', 'c', 15), ('t', 'd', 4)]


Résultat attendu:
[('a', 'b', 1),
 ('a', 's', 11),
 ('b', 'a', 3),
 ('b', 'c', 4),
 ('b', 'd', 3),
 ('b', 's', 8),
 ('c', 'a', 12),
 ('c', 'b', 5),
 ('c', 'd', 7),
 ('c', 't', 5),
 ('d', 'b', 11),
 ('s', 'a', 5),
 ('s', 'b', 5),
 ('t', 'c', 15),
 ('t', 'd', 4)]

2. Écrivez une fonction prenant en paramètres un graphe orienté pondéré et un flot maximum, et renvoyant une coupe minimum pour le graphe donné. Le flot est donné sous la forme d'un dictionnaire associant à chaque arc une valeur, et satisfait les propriétés d'un flot que vous ne devez donc pas vérifier. La coupe sera exprimée sous la forme d'un itérable contenant deux itérables de sommets. Votre fonction doit être en $O(|V|+|A|)$. Vous ne devez pas vérifier que le flot est bien maximum.

In [5]:
# corrigé exercice 1.2
def accessibles(graphe, depart, visites=None):
    resultat = set()
    a_traiter = {depart}  # n'importe quel ordre convient
    if visites is None:
        visites = {v: False for v in graphe.sommets()}
    while a_traiter:
        v = a_traiter.pop()
        if not visites[v]:
            visites[v] = True
            resultat.add(v)
            for w in graphe.successeurs(v):
                if not visites[w]:
                    a_traiter.add(w)
    
    return resultat


def coupe_minimum(graphe, flot_max, source, puits):
    S = accessibles(residuel(graphe, flot_max), source)
    return S, set(graphe.sommets()) - S

# tests -----------------------------------------------------------------------
from dictionnaireadjacenceorientepondere import DictionnaireAdjacenceOrientePondere as Graphe

# 1er exemple du cours
G = Graphe()
G.ajouter_arcs([
    ('s', 'a', 16), ('s', 'b', 13), ('a', 'c', 12), ('b', 'a', 4 ), 
    ('b', 'd', 14), ('c', 'b', 9 ), ('c', 't', 20), ('d', 'c', 7 ), 
    ('d', 't', 4 )
])
flot_max = {
    ('s', 'a'): 12, ('s', 'b'): 11, ('a', 'c'): 12, ('b', 'a'): 0, 
    ('b', 'd'): 11, ('c', 'b'): 0, ('c', 't'): 19, ('d', 'c'): 7, 
    ('d', 't'): 4 
}
coupe = coupe_minimum(G, flot_max, 's', 't')
print(sorted(map(sorted, coupe)))  # solution possible: [['a', 'b', 'd', 's'], ['c', 't']]
print(sum(G.poids_arc(u, v) for u in coupe[0] for v in coupe[1] if G.contient_arc(u, v)) == 23) # résultat attendu: True

[['a', 'b', 'd', 's'], ['c', 't']]
True


3. Écrivez une fonction prenant un flot en paramètre, ainsi que la source, et renvoyant sa valeur. Le flot est au même format que pour les questions précédentes, et il ne faut donc pas vérifier sa validité.

In [6]:
# corrigé exercice 1.3
def valeur(flot, source):
    return sum(flot[(u, v)] for u, v in flot if u == source)

# tests -----------------------------------------------------------------------
flot = {
    ('s', 'a'): 12, ('s', 'b'): 11, ('a', 'c'): 12, ('b', 'a'): 0, 
    ('b', 'd'): 11, ('c', 'b'): 0, ('c', 't'): 19, ('d', 'c'): 7, 
    ('d', 't'): 4 
}
print(valeur(flot, 's'))  # résultat attendu: 23

23


4. Écrivez une fonction qui renvoie la capacité d'une coupe pour le réseau donné. La coupe est donnée sous la forme d'un couple d'ensembles, dont les éléments sont les sommets. Votre algorithme doit être en $O(|V|+|A|)$.

In [7]:
def capacite(reseau, coupe):
    return sum(
        poids for u, v, poids in reseau.arcs() if u in coupe[0] and v in coupe[1]
    )
    
# tests -----------------------------------------------------------------------
from dictionnaireadjacenceorientepondere import DictionnaireAdjacenceOrientePondere as Graphe

# 1er exemple du cours
G = Graphe()
G.ajouter_arcs([
    ('s', 'a', 16), ('s', 'b', 13), ('a', 'c', 12), ('b', 'a', 4 ), 
    ('b', 'd', 14), ('c', 'b', 9 ), ('c', 't', 20), ('d', 'c', 7 ), 
    ('d', 't', 4 )
])
coupe = [{'s', 'a', 'b'}, {'c', 'd', 't'}]
print(capacite(G, coupe))  # résultat attendu: 26

26


5. Écrivez une fonction permettant de vérifier qu'une bipartition donnée est bien une coupe minimum. Pour ce faire, vous disposez du réseau de départ et d'un flot supposé maximum. Il faut donc vérifier que l'ensemble donné est bien une bipartition, qu'il s'agit d'une coupe, et que sa capacité est minimum. Votre fonction doit être en $O(|V|+|A|)$.

In [11]:
# corrigé exercice 1.5
def est_coupe_minimale(bipartition, graphe, source, puits, flot):
    # la bipartition est-elle une bipartition?
    S, T = bipartition
    if set(S).union(T) != set(graphe.sommets()):
        return False
    # la bipartition est-elle une coupe? source à "gauche", puits à "droite"
    if source not in S or puits not in T:
        return False
    # est-elle bien minimale? ssi sa capacité égale la valeur du flot
    valeur_flot = sum(flot[(source, v)] for v in graphe.successeurs(source))
    capacite_coupe = sum(
        graphe.poids_arc(u, v) for u in S for v in T
        if graphe.contient_arc(u, v)
    )
    return capacite_coupe == valeur_flot
    

# tests -----------------------------------------------------------------------
from dictionnaireadjacenceorientepondere import DictionnaireAdjacenceOrientePondere as Graphe

# 1er exemple du cours
G = Graphe()
G.ajouter_arcs([
    ('s', 'a', 16), ('s', 'b', 13), ('a', 'c', 12), ('b', 'a', 4 ), 
    ('b', 'd', 14), ('c', 'b', 9 ), ('c', 't', 20), ('d', 'c', 7 ), 
    ('d', 't', 4 )
])
flot_max = {
    ('s', 'a'): 12, ('s', 'b'): 11, ('a', 'c'): 12, ('b', 'a'): 0, 
    ('b', 'd'): 11, ('c', 'b'): 0, ('c', 't'): 19, ('d', 'c'): 7, 
    ('d', 't'): 4 
}
coupe = coupe_minimum(G, flot_max, 's', 't')
print(est_coupe_minimale(coupe, G, 's', 't', flot_max))  # résultat attendu: True
print(est_coupe_minimale([{'a'}, {'b'}], G, 's', 't', flot_max))  # résultat attendu: False (pas une coupe)
print(est_coupe_minimale([{'s', 'a'}, {'b', 'c', 'd', 't'}], G, 's', 't', flot_max))  # résultat attendu: False (coupe pas minimale)

True
False
False


## Exercice 2: composantes fortement connexes

1. Écrivez une fonction renvoyant les sommets d'un graphe orienté non pondéré dans l'ordre où le parcours en profondeur **termine** leur exploration. Votre fonction doit être en $O(|V|+|A|)$.


In [3]:
def parcours_profondeur_date(graphe):
    """Empile les sommets à la fin de leur exploration."""
    resultat = list()
    visites = {v: False for v in graphe.sommets()}
    def profondeur_rec(sommet):
        if not visites[sommet]:
            visites[sommet] = True
            for suivant in graphe.successeurs(sommet):
                profondeur_rec(suivant)
            resultat.append(sommet)
            
    for v in graphe.sommets():
        profondeur_rec(v)
        
    return resultat


# tests -----------------------------------------------------------------------
from dictionnaireadjacenceoriente import DictionnaireAdjacenceOriente as Graphe

G = Graphe()
G.ajouter_arcs(["01", "02", "12", "34", "56", "57"])
print(parcours_profondeur_date(G))  # résultat attendu: ['2', '1', '0', '4', '3', '7', '6', '5']


['2', '1', '0', '4', '3', '7', '6', '5']


2. Écrivez une fonction renvoyant un itérable contenant les composantes fortement connexes d'un graphe orienté. Chaque composante est représentée par les sommets qu'elle contient. La fonction renverra donc un itérable d'itérables. Votre fonction doit être en $O(|V|+|A|)$.


In [6]:
def renverser(graphe):
    """Renvoie le graphe renversé."""
    envers = type(graphe)()
    envers.ajouter_arcs((v, u) for u, v in graphe.arcs())
    return envers


def cfc(graphe):
    # passe 1: calculer l'ordre
    ordre_parcours = parcours_profondeur_date(graphe)[::-1]
    
    # passe 2 sur le graphe renversé
    cfc = list()
    envers = renverser(graphe)
    visites = {v: False for v in graphe.sommets()}
    for v in ordre_parcours:
        if not visites[v]:
            cfc.append(accessibles(envers, v, visites))
    return cfc

# tests -----------------------------------------------------------------------
from dictionnaireadjacenceoriente import DictionnaireAdjacenceOriente as Graphe

# exemple du cours
G = Graphe()
G.ajouter_arcs(["01", "12", "14", "15", "23", "26", "32", "37", "40", "45", "56", "65", "67"])
print(sorted(map(sorted, cfc(G)))) # résultat attendu: [['0', '1', '4'], ['2', '3'], ['5', '6'], ['7']]

[['0', '1', '4'], ['2', '3'], ['5', '6'], ['7']]

3. Écrivez une fonction prenant en paramètres un graphe orienté $G$ et ses composantes fortement connexes et renvoyant le graphe $H$ des composantes fortement connexes. Chaque composante fortement connexe de $G$ sera représentée dans $H$ par le sommet le plus petit qu'elle contient. Votre fonction doit être en $O(|V|+|A|)$.

In [14]:
def graphe_cfc(graphe, composantes):
    resultat = type(graphe)()
    # noter pour chaque sommet la cfc à laquelle il appartient
    partition = dict()
    for comp in composantes:
        identifiant = min(comp)
        for sommet in comp:
            partition[sommet] = identifiant
            
    # pour chaque arc (u, v) du graphe, rajouter au résultat l'arc (partition[u], partition[v])
    for u, v in graphe.arcs():
        if partition[u] != partition[v]:
            resultat.ajouter_arc(partition[u], partition[v])
        
    return resultat

# tests -----------------------------------------------------------------------
from dictionnaireadjacenceoriente import DictionnaireAdjacenceOriente as Graphe

# exemple du cours
G = Graphe()
G.ajouter_arcs(["01", "12", "14", "15", "23", "26", "32", "37", "40", "45", "56", "65", "67"])
H = graphe_cfc(G, cfc(G))
print(sorted(H.sommets()))  # résultat attendu: ['0', '2', '5', '7']
print(sorted(H.arcs()))  # résultat attendu: [('0', '2'), ('0', '5'), ('2', '5'), ('2', '7'), ('5', '7')]

['0', '2', '5', '7']
[('0', '2'), ('0', '5'), ('2', '5'), ('2', '7'), ('5', '7')]


## Exercice 3: plus courts chemins

1. Écrivez une fonction prenant en paramètres un graphe orienté pondéré dont les poids de tous les arcs sont 1, et un sommet de départ, et renvoyant l'arbre des plus courts chemins issus de ce sommet sous la forme d'un graphe orienté pondéré du même type que l'entrée. Votre fonction doit être en $O(|V|+|A|)$.

In [18]:
from collections import deque

def arbre_pcc(graphe, depart):
    resultat = type(graphe)()
    resultat.ajouter_sommet(depart)
    a_traiter = deque()
    a_traiter.extend((depart, w) for w in graphe.successeurs(depart))
    while a_traiter:
        v, w = a_traiter.popleft()
        if resultat.contient_sommet(v) and not resultat.contient_sommet(w):
            resultat.ajouter_arc(v, w, 1)
            for x in graphe.successeurs(w):
                if not resultat.contient_sommet(x):
                    a_traiter.append((w, x))
    
    return resultat

# tests -----------------------------------------------------------------------
from dictionnaireadjacenceorientepondere import DictionnaireAdjacenceOrientePondere as Graphe

G = Graphe()
G.ajouter_arcs([
    (0, 1, 1), (0, 2, 1), (1, 2, 1)
])
print(sorted(arbre_pcc(G, 0).arcs()))  # résultat attendu: [(0, 1, 1), (0, 2, 1)]
print(sorted(arbre_pcc(G, 2).arcs()))  # résultat attendu: []

[(0, 1, 1), (0, 2, 1)]
[]


2. Écrivez une fonction prenant en paramètres un graphe orienté pondéré dont les poids de tous les arcs sont 1 ou à 2, et un sommet de départ, et renvoyant l'arbre des plus courts chemins issus de ce sommet sous la forme d'un graphe orienté pondéré du même type que l'entrée. Votre fonction doit être en $O(|V|+|A|)$.

In [33]:
def arbre_pcc_1_ou_2(graphe, depart):
    # subdiviser les arcs de poids 2:
    prochain_indice = max(graphe.sommets()) + 1
    nouveaux_sommets = set()
    for u, v, poids in graphe.arcs():
        if poids == 2:
            graphe.retirer_arc(u, v)
            graphe.ajouter_arcs([(u, prochain_indice, 1), (prochain_indice, v, 1)])
            nouveaux_sommets.add(prochain_indice)
            prochain_indice += 1
            
    # utiliser l'algo précédent
    resultat = arbre_pcc(graphe, depart)
    
    # annuler les subdivisions dans le graphe de départ
    for x in nouveaux_sommets:
        u = next(iter(graphe.predecesseurs(x)))  # forcément unique
        v = next(iter(graphe.successeurs(x)))    # forcément unique
        graphe.ajouter_arc(u, v, 2)
        graphe.retirer_sommet(x)

    # annuler les subdivisions dans l'arbre; attention, il y a deux pièges:
    # 1) tous les nouveaux sommets n'appartiennent pas forcément à l'arbre
    # 2) ceux qui s'y trouvent peuvent ne pas avoir de successeur
    for x in nouveaux_sommets:
        if resultat.contient_sommet(x):
            u = next(iter(resultat.predecesseurs(x)))  # forcément unique
            if resultat.successeurs(x):  # n'existe pas nécessairement
                v = next(iter(resultat.successeurs(x)))    # mais forcément unique s'il existe
                resultat.ajouter_arc(u, v, 2)
            resultat.retirer_sommet(x)
    
    return resultat


# tests -----------------------------------------------------------------------
from dictionnaireadjacenceorientepondere import DictionnaireAdjacenceOrientePondere as Graphe

G = Graphe()
G.ajouter_arcs([
    (0, 1, 1), (0, 2, 2), (1, 2, 1), (1, 3, 1)
])
print(sorted(arbre_pcc_1_ou_2(G, 0).arcs()))  # résultat attendu: [(0, 1, 1), (1, 2, 1), (1, 3, 1)]
print(sorted(arbre_pcc_1_ou_2(G, 2).arcs()))  # résultat attendu: []

[(0, 1, 1), (1, 2, 1), (1, 3, 1)]
[]
